# `database.py`


In [1]:
# imports
import mysql.connector as sqltor


In [2]:
def connectToDb(username, password):
    dbCon = sqltor.connect(host='localhost', user=username,
                           passwd=password)
    if dbCon.is_connected() == True:
        return (dbCon, dbCon.cursor())
    else:
        print('> [ERROR] DB Connection error!\n')
        return (None, None)


In [64]:
#######################################################################
#                         DATABASE CREATION
#######################################################################

def createDb(dbCon, dbCursor, dbName='bank_db'):
    # create database if not already exists
    dbCursor.execute('CREATE DATABASE IF NOT EXISTS ' + dbName)
    dbCursor.execute('USE ' + dbName)


In [65]:
#######################################################################
#                         ACCOUNT TABLE CREATION
#######################################################################
# check accounts table
def createAccTable(dbCon, dbCursor, accountTableName='bank_accs'):
    createTableQuery = f'''CREATE TABLE IF NOT EXISTS
        {accountTableName} (
            accno       CHAR(10)    NOT NULL PRIMARY KEY,
            acctype     CHAR(4)     NOT NULL,
            accname     VARCHAR(20),
            maincustid  INT NOT NULL,
            jtcustid    INT,
            createdate  DATE,
            updatedate  DATE,
            balance     DECIMAL
        )
    '''
    dbCursor.execute(createTableQuery)
    dbCursor.execute('SELECT * FROM ' + accountTableName)
    ################## TBC????
    dbCursor.fetchall()
    
    return dbCursor.rowcount


In [66]:
def createCustTable(dbCon, dbCursor, customerTableName='bank_custs'):
    createTableQuery = f'''CREATE TABLE IF NOT EXISTS 
        {customerTableName} (
            custid      INT UNSIGNED NOT NULL AUTO_INCREMENT PRIMARY KEY,
            custname    VARCHAR(30),
            mobile      CHAR(10),
            address     VARCHAR(50)
        )
    '''
    dbCursor.execute(createTableQuery)
    dbCursor.execute('SELECT * FROM ' + customerTableName)
    ################## TBC????
    data = dbCursor.fetchall()
    
    return dbCursor.rowcount


In [86]:
def addCustToDb(dbCon, dbCursor, name, mobile, address):
    query = f'''
        INSERT INTO bank_custs(custname, mobile, address) VALUES(
            "{name}", "{mobile}", "{address}")
    '''
    dbCursor.execute(query)
    dbCon.commit()
    
    # dbCursor.rowcount will have number of rows inserted
    insertedRowCount = dbCursor.rowcount
    
    # dbCursor.lastrowid will have value generated for the 
    # AUTO_INCREMENT custid column
    newCustId = dbCursor.lastrowid
    
    return (insertedRowCount, newCustId)


In [112]:
def fetchCustomerDetailsFromDb(dbCursor, custid):
    query = f'SELECT * FROM bank_custs WHERE custid = {custid}'
    dbCursor.execute(query)
    data = dbCursor.fetchone()
    if (dbCursor.rowcount > 0):
        return data
    else:
        return None


In [134]:
def updateCustomerDetailsInDb(dbCon, dbCursor, 
                              custid, name, mobile, address):
    query = f'''
        UPDATE bank_custs SET 
            custname = "{name}", 
            mobile = "{mobile}", 
            address = "{address}"
        WHERE custid = {custid}
    '''
    dbCursor.execute(query)
    dbCon.commit()
    
    updatedRowCount = dbCursor.rowcount
        
    return updatedRowCount
    

In [136]:
def deleteCustomerFromDb(dbCon, dbCursor, custid):
    query = f'DELETE FROM bank_custs WHERE custid = {custid}'
    dbCursor.execute(query)
    dbCon.commit()
    
    deletedRowCount = dbCursor.rowcount
        
    return deletedRowCount
    

In [145]:
def getCustomerCountFromDb(dbCursor):
    dbCursor.execute('SELECT * FROM bank_custs')
    dbCursor.fetchall()
    return dbCursor.rowcount


In [146]:
def getAccCountFromDb(dbCursor):
    dbCursor.execute('SELECT * FROM bank_accs')
    dbCursor.fetchall()
    return dbCursor.rowcount


In [173]:
def addAccToDb(dbCon, dbCursor, accNo, accType, accName,
               primCustId, jtCustId, createDate, updateDate, balance):
    query = f'''
        INSERT INTO bank_accs(accno, acctype, accname, maincustid, jtcustid,
            createdate, updatedate, balance) VALUES(
            "{accNo}", "{accType}", "{accName}", 
            {primCustId}, {jtCustId}, 
            "{createDate}", "{updateDate}", {balance})
    '''
    dbCursor.execute(query)
    dbCon.commit()
    
    # dbCursor.rowcount will have number of rows inserted
    insertedRowCount = dbCursor.rowcount
        
    return insertedRowCount


In [162]:
def fetchAccDetailsFromDb(dbCursor, accNo):
    query = f'SELECT * FROM bank_accs WHERE accno = "{accNo}"'
    dbCursor.execute(query)
    data = dbCursor.fetchone()
    if (dbCursor.rowcount > 0):
        return data
    else:
        return None

In [179]:
def deleteAccFromDb(dbCon, dbCursor, accNo):
    query = f'DELETE FROM bank_accs WHERE accno = "{accNo}"'
    dbCursor.execute(query)
    dbCon.commit()
    
    deletedRowCount = dbCursor.rowcount
        
    return deletedRowCount
    

In [193]:
def updateAccInDb(dbCon, dbCursor, accNo, balance, updateDt):
    query = f'''
        UPDATE bank_accs SET 
            balance = {balance}, 
            updatedate = "{updateDt}"
        WHERE accno = "{accNo}"
    '''
    dbCursor.execute(query)
    dbCon.commit()
    
    updatedRowCount = dbCursor.rowcount
        
    return updatedRowCount
    

In [28]:
def closeDbConnection(dbCon, dbCursor):
    dbCursor.fetchall()
    dbCursor.close()
    dbCon.close()
    return (None, None)


# `menu.py`

In [8]:
#######################################################################
#                           DISPLAY MENU
#######################################################################

def displayMenu():
    print('+--------------------------------------------+')
    print('| BANK OPERATION MENU                        |')
    print('|                                            |')
    print('|   Available operations                     |')
    print('|                                            |')
    print('|   1. Create customer                       |')
    print('|   2. Get customer details                  |')
    print('|   3. Update customer details               |')
    print('|   4. Delete customer                       |')
    print('|   5. Open bank account                     |')
    print('|   6. Get account details                   |')
    print('|   7. Delete bank account                   |')
    print('|   8. Transact                              |')
    print('|   9. Quit                                  |')
    print('+--------------------------------------------+\n')    


# `main.py`

In [75]:
def getCustInfo():
    print('ENTER CUSTOMER DETAILS')
    name = input('  Customer name? ')
    mobile = input('  Mobile number? ')
    address = input('  Address? ')
    
    return (name, mobile, address)

In [90]:
def newCustomer(dbCon, dbCursor):
    (name, mobile, address) = getCustInfo()
    (insertedRowCount, newCustId) = addCustToDb(
        dbCon, dbCursor, name, mobile, address)
    print(insertedRowCount, 'new customer added. custid:', newCustId)
    

In [119]:
def getCustomerDetails(dbCursor):
    custid = input('Enter custid: ')
    data = fetchCustomerDetailsFromDb(dbCursor, custid)
    if data != None:
        print('CUSTOMER DETAILS')
        print('(custid, custname, mobile, address)')
        print(data)
    else:
        print('Customer NOT found! custid =', custid)

    

In [133]:
def updateCustomerDetails(dbCon, dbCursor):
    custid = input('Enter custid to update: ')
    # get existing details
    existingData = fetchCustomerDetailsFromDb(dbCursor, custid)
    if existingData != None:
        print('Enter updated information, leave empty if no change...')
        newName = input('  New name? ')
        if newName == '':
            newName = existingData[1]
        newMobile = input('  New mobile number? ')
        if newMobile == '':
            newMobile = existingData[2]
        newAddress = input('  New address? ')
        if newAddress == '':
            newAddress = existingData[3]
        count = updateCustomerDetailsInDb(dbCon, dbCursor, custid,
                                          newName, newMobile,
                                          newAddress)
        if count == 1:
            print('Customer details update successful.')
        else:
            print('[ERROR] Customer details update failed!')
    else:
        print('Customer NOT found! custid =', custid)


In [141]:
def deleteCustomer(dbCon, dbCursor):
    custid = input('Enter custid to delete: ')
    count = deleteCustomerFromDb(dbCon, dbCursor, custid)
    if count == 1:
        print('Deleted customer, custid:', custid)
    else:
        print('[ERROR] Delete customer failed! custid:', custid)


In [158]:
def openBankAcc(dbCon, dbCursor):
    print('CREATING A NEW BANK ACCOUNT')
    
    # Choose account type
    accType = "SBAC"
    prefix = 'S'
    print('\n  Step 1 - Account Type Selection')
    print('  1. Savings\n  2. Current\n  3. Fixed deposit)\n')
    typeInp = int(input('Choose account type (1/2/3): '))
    if typeInp == 1:
        accType = "SBAC"
        accName = 'Savings Account'
        prefix = 'S'
    elif typeInp == 2:
        accType = "CUAC"
        accName = 'Current Account'
        prefix = 'C'
    elif typeInp == 3:
        accType = "FDAC"
        accName = 'Fixed Account'
        prefix = 'F'
    else:
        print('> [ERROR] Failed! Wrong account type ({typeInp})!')
        return False
    
    # Main customer details
    print('\n  Step 2 - Primary Customer Selection')
    primCustId = input('  Enter custid of primary Customer: ')
    data = fetchCustomerDetailsFromDb(dbCursor, primCustId)
    if data == None:
        print('[ERROR] Failed to open acc. No such primary customer!')
        return False
    
    # Joint customer details
    print('\n  Step 3 - Joint Customer Selection')
    isJtAcc = input('  Any joint customer (y/n)? ')
    if isJtAcc == 'y':
        jtCustId = input('  Enter custid of joint Customer: ')
        data = fetchCustomerDetailsFromDb(dbCursor, jtCustId)
        if data == None:
            print('[ERROR] Failed to open acc. No such joint customer!')
            return False
    else:
        jtCustId = primCustId
    
    # Creation date
    print('\n  Step 4 - Enter creation date')
    createDate = input('  Create date (YYYY-MM-DD)? ')
    
    # Initial balance
    balance = float(input(' Enter initial account balance: '))
    
    # Make a unique account no based on number of existing accounts
    accCount = getAccCountFromDb(dbCursor)
    accNo = prefix + str(accCount + 1)
    
    insertedRowCount = addAccToDb(dbCon, dbCursor, 
                                  accNo, accType, accName,
                                  primCustId, jtCustId,
                                  createDate, createDate, balance)
    
    if insertedRowCount == 1:
        print('Successfully opened account.')
        return True
    else:
        print('[ERROR] Failed to open account.')
        return False


In [166]:
def getAccDetails(dbCursor):
    accNo = input('Enter accNo: ')
    data = fetchAccDetailsFromDb(dbCursor, accNo)
    if data != None:
        print('ACCOUNT DETAILS')
        print('(accno, acctype, accname, maincustid, jtcustid, '+
            'createdate, updatedate, balance)')
        print(data)
    else:
        print('Account NOT found! accNo =', accNo)


In [181]:
def deleteAcc(dbCon, dbCursor):
    accNo = input('Enter accNo to delete: ')
    count = deleteAccFromDb(dbCon, dbCursor, accNo)
    if count == 1:
        print('Successfully deleted account.')
    else:
        print('[ERROR] Failed to delete account! accNo =', accNo)
    

In [191]:
def transact(dbCon, dbCursor):
    accNo = input('Enter accNo to transact: ')
    data = fetchAccDetailsFromDb(dbCursor, accNo)
    if data != None:
        print('ACCOUNT DETAILS')
        print('(accno, acctype, accname, maincustid, jtcustid, '+
            'createdate, updatedate, balance)')
        print(data)
        type = int(input('Select type of transaction ' + 
                        '(1. Withdraw 2. Deposit): '))
        amount = float(input('Enter amount: '))
        balance = float(data[7])
        if type == 1:
            if amount > data[7]:
                print('[ERROR] Withdraw error! Insufficient balance.')
                return False
            else:
                balance -= amount
        elif type == 2:
            balance += amount
            
        updateDt = input('  Update date (YYYY-MM-DD)? ')
        count = updateAccInDb(dbCon, dbCursor, accNo, balance, updateDt)
        if count == 1:
            print('Transaction successful.')
            return True
        else:
            print('[ERROR] Transaction failed!')
            return False       
    else:
        print('Account NOT found! accNo =', accNo)
        return False
        

In [198]:
# print a banner
print('**************************')
print('* BANK MANAGEMENT SYSTEM *')
print('**************************')


**************************
* BANK MANAGEMENT SYSTEM *
**************************


In [199]:
CORRECT_USER_NAME = 'maharnab'
CORRECT_PASSWORD = 'maharnab'


In [200]:
# Login to the system
print('ENTER CREDENTIALS TO LOG IN TO THE SYSTEM')
print('(Maximum 3 login attempts are allowed)\n')

userName = ''
password = ''

# Should fail after 3 login attempts
for i in range(3):
    userName = input('Enter the user name: ')
    password = input(f'Enter the password for {userName}: ')
    # validate entered username and password
    if userName != CORRECT_USER_NAME or password != CORRECT_PASSWORD:
        print('> ERROR: Wrong user name or password!')
        print('> Remaining', 3-i-1, 'attempts!')
    else:
        print('> Credentials verified successfully...')
        break
else:
    print('> [ERROR] Failed to login! EXIT!')
    ######################## UNCOMMENT ########################
    # quit()


ENTER CREDENTIALS TO LOG IN TO THE SYSTEM
(Maximum 3 login attempts are allowed)

> Credentials verified successfully...


In [201]:
# connect to mysql
print('> Initiailizing bank management system...')
print('> Logging into the database...')
dbCon, dbCursor = connectToDb(userName, password)
if dbCon == None:
    print('> [ERROR] Failed to log into the database. EXIT')
    # quit
else:
    print('> Successfully logged into the database...')


> Initiailizing bank management system...
> Logging into the database...
> Successfully logged into the database...


In [202]:
createDb(dbCon, dbCursor)


In [203]:
# check accounts table
print('> Checking accounts table...')
accountTableName = 'bank_accs'
existingAccs = createAccTable(dbCon, dbCursor) 
print(f'> Found details of {existingAccs} accounts...')

# check customer table
print('> Checking customer table...')
customerTableName='bank_custs'
existingCusts = createCustTable(dbCon, dbCursor) 
print(f'> Found details of {existingCusts} customers...')


> Checking accounts table...
> Found details of 0 accounts...
> Checking customer table...
> Found details of 0 customers...


In [204]:
choice = 0
while choice != 9:
    displayMenu()
    choice = int(input('>>> Enter your choice [1-9]:'))
    if choice == 1:
        newCustomer(dbCon, dbCursor)
    elif choice == 2:
        getCustomerDetails(dbCursor)
    elif choice == 3:
        updateCustomerDetails(dbCon, dbCursor)
    elif choice == 4:
        deleteCustomer(dbCon, dbCursor)
    elif choice == 5:
        openBankAcc(dbCon, dbCursor)
    elif choice == 6:
        getAccDetails(dbCursor)
    elif choice == 7:
        deleteAcc(dbCon, dbCursor)
    elif choice == 8:
        transact(dbCon, dbCursor)
    elif choice == 9:
        print('Bye!')
        closeDbConnection(dbCon, dbCursor)
    else:
        print('[ERROR] Invalid choice. Select right operation.')
        choice = 0


+--------------------------------------------+
| BANK OPERATION MENU                        |
|                                            |
|   Available operations                     |
|                                            |
|   1. Open bank account                     |
|   2. Get account details                   |
|   3. Delete bank account                   |
|   4. Transact                              |
|   5. Get customer details                  |
|   6. Update customer details               |
|   7. Delete customer                       |
|   8. Clear screen and show menu            |
|   9. Quit                                  |
+--------------------------------------------+

Bye!
